# **KDDCup Data Analytics with PySpark RDD: A structured case study**

### Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark

### Author: Amin Karami (PhD, FHEA)

##### data source: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html


In [ ]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

In [1]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [2]:
from pyspark import SparkContext, SparkConf

# Initializing Spark
conf = SparkConf().setAppName("KDDCup_PySpark").setMaster("local[*]")
sc = SparkContext(conf=conf)
print(sc)
print("Ready to go!")

23/03/20 08:32:14 WARN Utils: Your hostname, Adrian-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.100.19 instead (on interface en0)
23/03/20 08:32:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 08:32:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<SparkContext master=local[*] appName=KDDCup_PySpark>
Ready to go!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
# Read and Load Data to Spark
# Data source: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
rdd = sc.textFile('kddcup.data.gz')


In [8]:
# Repartition and Cache Data:
rdd = rdd.repartition(10)
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

rdd.persist()

8
10


MapPartitionsRDD[16] at coalesce at NativeMethodAccessorImpl.java:0

## Question 1: Get ten records randomly


In [9]:
rdd.takeSample(False, 10, 1234)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '3327,udp,other,SF,146,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,2,0.01,0.67,0.95,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,278,19,1.00,1.00,0.00,0.00,0.07,0.05,0.00,255,19,0.07,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,eco_i,SF,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,26,0.00,0.00,0.00,0.00,1.00,0.00,1.00,2,255,1.00,0.00,1.00,0.50,0.00,0.00,0.00,0.00,ipsweep.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103,5,1.00,1.00,0.00,0.00,0.05,0.06,0.00,255,18,0.07,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,icmp,ecr_i,SF,520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.

## Question 2: Count elements

In [10]:
rdd.count()

4898431

## Question 3: Calculate the ratio of `normal` connections


In [13]:
Normal_rdd = rdd.filter(lambda line: 'normal' in line)

ratio = Normal_rdd.count() / rdd.count()

print(" the ratio of normal connection is {} %".format(round(ratio,4)*100))

 the ratio of normal connection is 19.86 %


## Question 4: Get the list of labels


In [15]:
Split_rdd = rdd.map(lambda line: line.split(","))

label_rdd = Split_rdd.map(lambda item: item[-1]).distinct()

label_rdd.collect()



['neptune.',
 'loadmodule.',
 'warezclient.',
 'pod.',
 'smurf.',
 'nmap.',
 'spy.',
 'back.',
 'teardrop.',
 'ipsweep.',
 'multihop.',
 'phf.',
 'ftp_write.',
 'guess_passwd.',
 'normal.',
 'land.',
 'satan.',
 'imap.',
 'portsweep.',
 'warezmaster.',
 'rootkit.',
 'buffer_overflow.',
 'perl.']

## Question 5: Count the number of connections for each label

In [16]:
rdd.filter(lambda line: 'neptune.' in line).count()

def LabelCount_func(items):
    Labels_Count = []

    for i in items:
        Labels_Count.append(rdd.filter(lambda line: i in line).count())
    return Labels_Count



In [18]:
%%time
summary_labels = LabelCount_func(label_rdd.collect())

CPU times: user 209 ms, sys: 65.2 ms, total: 274 ms
Wall time: 41.9 s


None


In [19]:
%%time
label_rdd_KV = Split_rdd.map(lambda x : (x[-1],1))
label_rdd_reduce = label_rdd_KV.reduceByKey(lambda a,b : a+b)

CPU times: user 6.01 ms, sys: 3.18 ms, total: 9.19 ms
Wall time: 23 ms


In [21]:
import pandas as pd

Keys = label_rdd_reduce.keys().collect()
values = label_rdd_reduce.values().collect()


DF_labels_KV = pd.DataFrame({
    'Label': Keys,
    'Count': values
})

DF_labels_KV.sort_values(by="Count", ascending=False)

,Label,Count
4,smurf.,2807886
0,neptune.,1072017
14,normal.,972781
16,satan.,15892
9,ipsweep.,12481
18,portsweep.,10413
5,nmap.,2316
7,back.,2203
2,warezclient.,1020
8,teardrop.,979


## Question 6: Get the connection type with successful `root_shell` connections to servers, where the number of data bytes from source (`src_bytes`) is 500 times more than from server (`dst_bytes`)

In [26]:
Split_rdd.filter(lambda x: x[13] == '1')\
    .map(lambda x: (x[1], x[4], x[5]))\
    .filter(lambda x: int(x[2]) > int(x[1])* 500)\
    .collect()

[('tcp', '353', '759161'),
 ('tcp', '433', '1524348'),
 ('tcp', '296', '507534'),
 ('tcp', '296', '507534'),
 ('tcp', '246', '866032'),
 ('tcp', '317', '394616'),
 ('tcp', '262', '744605'),
 ('tcp', '173', '744605'),
 ('tcp', '224', '2776333'),
 ('tcp', '262', '744605'),
 ('tcp', '0', '2072'),
 ('tcp', '351', '759161'),
 ('tcp', '1794', '3851730'),
 ('tcp', '465', '320362'),
 ('tcp', '0', '2072'),
 ('tcp', '0', '2072'),
 ('tcp', '296', '507534'),
 ('tcp', '266', '507534'),
 ('tcp', '255', '574784'),
 ('tcp', '0', '2072')]

## Question 7:  Get the list of `Protocols`that are `normal` and `vulnerable to attacks`, where there is NOT `guest login` to the destination addresses


In [31]:
normal_protocols_rdd = Split_rdd.filter(lambda line: "normal" in line[-1] and line[21] != '1')\
                        .map(lambda line: (line[1],1)).reduceByKey(lambda a,b: a+b)

attack_protocols_rdd = Split_rdd.filter(lambda line: "normal" not in line[-1] and line[21] != '1')\
                        .map(lambda line: (line[1],1)).reduceByKey(lambda a,b: a+b)

normal_KeyValue = pd.DataFrame({
    'label': normal_protocols_rdd.keys().collect(),
    'State': 'normal',
    'Count': normal_protocols_rdd.values().collect()
})

attack_KeyValue = pd.DataFrame({
    'label': attack_protocols_rdd.keys().collect(),
    'State': 'attack',
    'Count': attack_protocols_rdd.values().collect()
})


results = normal_KeyValue.append(attack_KeyValue)

results.sort_values(by='label', ascending=False)

/var/folders/0w/_k96360175sff4w3b2nb6ts40000gn/T/ipykernel_1172/2383834916.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = normal_KeyValue.append(attack_KeyValue)


,label,State,Count
1,udp,normal,191348
1,udp,attack,2940
2,tcp,normal,764894
2,tcp,attack,1101613
0,icmp,normal,12763
0,icmp,attack,2820782


## Question 8: Get a summary statistics for the sum of `tcp` connections to the same destination IP address (hint: `protocol_type` and `dst_host_count` features)

In [33]:
# Source: https://spark.apache.org/docs/latest/mllib-statistics.html
from pyspark.mllib.stat import Statistics
from math import sqrt

summary = Statistics.colStats(Split_rdd.filter(lambda line: line[1] == 'tcp').map(lambda line: [int(line[31])] ))

tcp_mean = round(float(summary.mean()),3)
tcp_std = round(float(sqrt(summary.variance())),3)
tcp_min = round(float(summary.min()),3)
tcp_max = round(float(summary.max()),3)

print([tcp_mean, tcp_std, tcp_min, tcp_max])





[201.752, 90.726, 0.0, 255.0]


## [challenge] Question 9: Filter the number of `icmp`-based attacks for each `service`